In [1]:
from fastapi import FastAPI, Depends, HTTPException, Query
from sqlalchemy.orm import Session
from sqlalchemy import desc, func
from typing import List
from database import SessionLocal
from table_user import User
from table_post import Post
from table_feed import Feed
from schema import UserGet, PostGet, FeedGet


app = FastAPI()



def get_db():
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


@app.get("/user/{id}", response_model=UserGet)
def get_user(id: int, db: Session = Depends(get_db)):
    user = db.query(User).filter(User.id == id).first()
    if user is None:
        raise HTTPException(status_code=404, detail="user not found")
    return user


@app.get("/post/{id}", response_model=PostGet)
def get_post(id: int, db: Session = Depends(get_db)):
    post = db.query(Post).filter(Post.id == id).first()
    if post is None:
        raise HTTPException(status_code=404, detail="post not found")
    return post


@app.get("/user/{id}/feed", response_model=List[FeedGet])
def get_user_feed(id: int, limit: int = Query(10, gt=0), db: Session = Depends(get_db)):
    feed = (
        db.query(Feed)
        .filter(Feed.user_id == id)
        .order_by(desc(Feed.time))
        .limit(limit)
        .all()
    )
    return feed


@app.get("/post/{id}/feed", response_model=List[FeedGet])
def get_post_feed(id: int, limit: int = Query(10, gt=0), db: Session = Depends(get_db)):
    feed = (
        db.query(Feed)
        .filter(Feed.post_id == id)
        .order_by(desc(Feed.time))
        .limit(limit)
        .all()
    )
    return feed

@app.get("/post/recommendations/", response_model=List[PostGet])
def get_post_recommendations(limit: int = Query(10, gt=0), id: int = Query(...), db: Session = Depends(get_db)):
    subquery = (
        db.query(Feed.post_id, func.count(Feed.post_id).label("like_count"))
        .filter(Feed.action == "like")
        .group_by(Feed.post_id)
        .order_by(desc("like_count"))
        .limit(limit)
        .subquery()
    )

    top_posts = (
        db.query(Post)
        .join(subquery, Post.id == subquery.c.post_id)
        .all()
    )

    return top_posts

ModuleNotFoundError: No module named 'database'